In [ ]:
#@title # ⚙️ 1.Cấu hình cài đặt ComfyUI
#@markdown

import os
import subprocess
import zipfile
import threading
import time
import socket
import shutil
import tarfile
from tqdm import tqdm  # Hiển thị tiến trình tải

# === Cấu hình thiết lập ===
GITHUB_URL = "https://github.com/comfyanonymous/ComfyUI"
CLOUDFLARE_DEB = "cloudflared-linux-amd64.deb"
TUNNELTO_PATH = "/root/.tunnelto/bin/tunnelto"
COMFYUI_PORT = 8188
# Biến kiểm soát tiến trình Tunnelto
stop_tunnelto = False

#CONFIG_DIR = '/content/TAConfig'

#!git clone https://github.com/sipivn/TA-ComfyUI {CONFIG_DIR}

# Xác định nơi cài đặt
if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_DIR = "/content/drive/MyDrive/ComfyUI"
    #CONFIG_DIR = os.path.join(BASE_DIR, "TAConfig")
else:
    BASE_DIR = "/content/ComfyUI"
    #CONFIG_DIR = BASE_DIR  # Nếu không dùng Google Drive, cài trên Google Colab

# Các đường dẫn đến file danh sách link tải
MODEL_LINKS_FILE = os.path.join(CONFIG_DIR, "model_links.txt")
VAE_LINKS_FILE = os.path.join(CONFIG_DIR, "vae_links.txt")
CONTROLNET_LINKS_FILE = os.path.join(CONFIG_DIR, "controlnet_links.txt")
CLIP_LINKS_FILE = os.path.join(CONFIG_DIR, "clip_links.txt")
UPSCALE_LINKS_FILE = os.path.join(CONFIG_DIR, "upscale_links.txt")
CUSTOM_NODES_FILE = os.path.join(CONFIG_DIR, "custom_nodes.txt")

# Các thư mục lưu model tải về
MODEL_DIR = os.path.join(BASE_DIR, "models/checkpoints")
VAE_DIR = os.path.join(BASE_DIR, "models/vae")
CONTROLNET_DIR = os.path.join(BASE_DIR, "models/controlnet")
CLIP_DIR = os.path.join(BASE_DIR, "models/clip")
UPSCALE_DIR = os.path.join(BASE_DIR, "models/upscale_models")
CUSTOM_NODES_DIR = os.path.join(BASE_DIR, "custom_nodes")

# Kiểm tra & tạo thư mục nếu chưa tồn tại
os.makedirs(BASE_DIR, exist_ok=True)

def install_aria2c():
    """Cài đặt aria2c nếu chưa có."""
    try:
        subprocess.run(["aria2c", "--version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        print("✅ aria2c đã được cài đặt!")
    except FileNotFoundError:
        print("⚠️ aria2c chưa được cài đặt! Tiến hành cài đặt...")
        subprocess.run(["apt-get", "install", "-y", "aria2"], check=True)
        print("✅ Cài đặt aria2c thành công!")

def install_comfyui():
    """Cài đặt hoặc cập nhật ComfyUI từ GitHub."""
    print(f"\n🔍 Cài đặt ComfyUI vào {'Google Drive' if USE_GOOGLE_DRIVE else 'Google Colab'}...")

    # Kiểm tra xem Git đã được cài đặt chưa
    try:
        subprocess.run(["git", "--version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
    except FileNotFoundError:
        print("❌ Git chưa được cài đặt! Vui lòng cài đặt Git trước khi tiếp tục.")
        return

    if not os.path.exists(BASE_DIR) or not os.listdir(BASE_DIR):
        print("🚀 Cloning ComfyUI repository...")
        result = subprocess.run(["git", "clone", "--depth=1", GITHUB_URL, BASE_DIR], text=True, capture_output=True)

        if result.returncode == 0:
            print("✅ Clone ComfyUI hoàn tất!")
        else:
            print(f"❌ Lỗi khi clone ComfyUI: {result.stderr}")
            return
    else:
        print("✅ ComfyUI đã có sẵn!")

    os.chdir(BASE_DIR)

    if UPDATE_COMFY_UI:
        print("\n🔄 Đang cập nhật ComfyUI...")
        result = subprocess.run(["git", "pull"], text=True, capture_output=True)

        if result.returncode == 0:
            print("✅ Cập nhật ComfyUI hoàn tất!")
        else:
            print(f"❌ Lỗi khi cập nhật ComfyUI: {result.stderr}")

# === Cài đặt dependencies CUDA ===
def install_cuda_dependencies():
    print("\n🔍 Kiểm tra phiên bản CUDA...")
    cuda_versions = ["cu121", "cu118", "cu117"]
    selected_cuda = None

    for version in cuda_versions:
        test_url = f"https://download.pytorch.org/whl/{version}"
        print(f"🔎 Thử cài đặt với CUDA {version}...")
        result = subprocess.run(["pip", "install", "--extra-index-url", test_url, "torch"], capture_output=True, text=True)

        if "ERROR" not in result.stderr:
            selected_cuda = version
            print(f"✅ Chọn phiên bản CUDA: {selected_cuda}")
            break
    else:
        print("⚠️ Không xác định được phiên bản CUDA, dùng mặc định.")

    print(f"🚀 Đang cài đặt dependencies với CUDA {selected_cuda}...")
    subprocess.run([
        "pip", "install", "xformers!=0.0.18", "-r", "requirements.txt",
        "--extra-index-url", f"https://download.pytorch.org/whl/{selected_cuda}"
    ], check=True)
    print("✅ Cài đặt dependencies hoàn tất!")


def clone_repo(link, destination):
    """Clone một repository GitHub vào thư mục chỉ định bằng subprocess."""
    
    if not link.startswith("https://github.com/"):
        print(f"❌ Link không hợp lệ: {link}")
        return

    # Tạo thư mục đích nếu chưa có
    os.makedirs(destination, exist_ok=True)

    print(f"🚀 Đang clone repository từ: {link} vào {destination}...")

    # Chạy lệnh git clone bằng subprocess
    result = subprocess.run(["git", "clone", link, destination], text=True, capture_output=True)

    if result.returncode == 0:
        print(f"✅ Clone thành công vào: {destination}")
    else:
        print(f"❌ Lỗi khi clone repository: {result.stderr}")


# === Hàm tải file bằng aria2c ===
def download_files(file_path, output_dir):
    """Tải file từ danh sách link có trong file txt bằng aria2c. Phân tách URL & tên file bằng '@='."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if not os.path.exists(file_path):
        print(f"❌ Không tìm thấy file danh sách: {file_path}")
        return

    with open(file_path, "r") as f:
        links = f.readlines()

    for line in tqdm(links, desc="📥 Đang tải", unit="file"):
        line = line.strip()
        if not line:
            continue

        try:
            url, filename = line.split("@=")  # Phân tách URL và tên file
        except ValueError:
            print(f"❌ Lỗi định dạng dòng: {line}")
            continue

        output_path = os.path.join(output_dir, filename)
        if os.path.exists(output_path):
            print(f"✅ Đã tồn tại: {filename}, bỏ qua...")
        else:
            subprocess.run(["aria2c", "-c", "-x", "16", "-s", "16", "-k", "1M", "-d", output_dir, "-o", filename, url])

# === Hàm tải riêng từng loại model ===
def install_checkpoints():
    download_files(MODEL_LINKS_FILE, MODEL_DIR)

def install_vae():
    download_files(VAE_LINKS_FILE, VAE_DIR)

def install_controlnet():
    download_files(CONTROLNET_LINKS_FILE, CONTROLNET_DIR)

def install_clip():
    download_files(CLIP_LINKS_FILE, CLIP_DIR)

def install_upscale_models():
    download_files(UPSCALE_LINKS_FILE, UPSCALE_DIR)

# === Cài đặt Custom Nodes ===
def install_custom_nodes(file):
    """Cài đặt Custom Nodes từ danh sách GitHub trong file txt (đường dẫn do người dùng tự nhập)."""

    # Kiểm tra file có tồn tại không
    if not os.path.isfile(file):
        print(f"❌ Không tìm thấy file danh sách Custom Nodes: {file}")
        return

    # Tạo thư mục chứa Custom Nodes nếu chưa có
    os.makedirs(CUSTOM_NODES_DIR, exist_ok=True)

    # Đọc danh sách Custom Nodes từ file
    with open(file, "r", encoding="utf-8") as f:
        nodes = f.readlines()

    # Di chuyển vào thư mục Custom Nodes để clone
    os.chdir(CUSTOM_NODES_DIR)

    for node_url in nodes:
        node_url = node_url.strip()
        if not node_url:
            continue

        repo_name = node_url.split("/")[-1]  # Lấy tên repository từ URL
        node_dir = os.path.join(CUSTOM_NODES_DIR, repo_name)

        if os.path.exists(node_dir):
            print(f"✅ Custom Node '{repo_name}' đã tồn tại, bỏ qua...")
            continue

        print(f"🚀 Đang cài đặt Custom Node: {repo_name}")
        result = subprocess.run(["git", "clone", node_url, node_dir], text=True, capture_output=True)

        if result.returncode == 0:
            print(f"✅ Clone hoàn tất: {repo_name}")
        else:
            print(f"❌ Lỗi khi clone {repo_name}: {result.stderr}")
            continue

        # Kiểm tra & cài đặt dependencies nếu có
        os.chdir(node_dir)
        req_file = os.path.join(node_dir, "requirements.txt")

        if os.path.exists(req_file):
            print(f"📦 Cài đặt dependencies cho {repo_name}...")
            subprocess.run(["pip", "install", "-r", req_file], text=True)

        print(f"✅ Hoàn tất cài đặt Custom Node: {repo_name}")

        # Quay lại thư mục chứa Custom Nodes để tiếp tục cài đặt Node tiếp theo
        os.chdir(CUSTOM_NODES_DIR)

    # Quay lại thư mục gốc
    os.chdir(BASE_DIR)
    print("\n🏠 Quay lại thư mục BASE_DIR sau khi hoàn tất cài đặt Custom Nodes.")


def install_custom_nodes_link(link):
    """Cài đặt một Custom Node từ link GitHub."""

    if not link.startswith("https://github.com/"):
        print(f"❌ Link không hợp lệ: {link}")
        return

    # Lấy tên repository từ link GitHub
    repo_name = link.rstrip("/").split("/")[-1]
    node_dir = os.path.join(CUSTOM_NODES_DIR, repo_name)

    # Tạo thư mục chứa Custom Nodes nếu chưa có
    os.makedirs(CUSTOM_NODES_DIR, exist_ok=True)

    # Kiểm tra nếu Node đã tồn tại
    if os.path.exists(node_dir):
        print(f"✅ Custom Node '{repo_name}' đã tồn tại, bỏ qua...")
        return

    print(f"🚀 Đang cài đặt Custom Node: {repo_name}")

    # Clone repository từ GitHub
    result = subprocess.run(["git", "clone", link, node_dir], text=True, capture_output=True)

    if result.returncode == 0:
        print(f"✅ Clone hoàn tất: {repo_name}")
    else:
        print(f"❌ Lỗi khi clone {repo_name}: {result.stderr}")
        return

    # Kiểm tra & cài đặt dependencies nếu có
    os.chdir(node_dir)
    req_file = os.path.join(node_dir, "requirements.txt")

    if os.path.exists(req_file):
        print(f"📦 Cài đặt dependencies cho {repo_name}...")
        subprocess.run(["pip", "install", "-r", req_file], text=True)

    print(f"✅ Hoàn tất cài đặt Custom Node: {repo_name}")

    # Quay lại thư mục gốc
    os.chdir(BASE_DIR)
    print("🏠 Quay lại thư mục BASE_DIR.")

def tunnelto_thread(port, api):
    """Cài đặt Tunnelto nếu chưa có và khởi chạy Tunnelto."""

    # Kiểm tra xem Tunnelto đã được cài đặt chưa
    if not os.path.exists(TUNNELTO_PATH):
        print("\n🌐 Tunnelto chưa được cài đặt, đang tiến hành cài đặt...")
        os.system("curl -sL https://tunnelto.dev/install.sh | sh")
        print("✅ Cài đặt Tunnelto hoàn tất!")

    # Chờ cho đến khi ComfyUI mở cổng `port`
    print(f"\n⏳ Đang chờ ComfyUI khởi động trên cổng {port}...")
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print(f"\n✅ ComfyUI đã khởi động! Đang mở Tunnelto trên cổng {port}...\n")

    # Thiết lập API key và chạy Tunnelto
    os.system(f"{TUNNELTO_PATH} set-auth --key {api[0]}")
    os.system(f"{TUNNELTO_PATH} --subdomain {api[1]} --port {port} &")

    print(f"🔗 Tunnelto đã hoạt động! Truy cập ComfyUI tại: https://{api[1]}.tunn.dev")

def tunnelto_thread_background(port, api):
    """Cài đặt & chạy Tunnelto, có thể dừng khi cần."""

    global stop_tunnelto  # Để có thể kiểm soát tiến trình

    # Kiểm tra xem Tunnelto đã được cài đặt chưa
    if not os.path.exists(TUNNELTO_PATH):
        print("\n🌐 Tunnelto chưa được cài đặt, đang tiến hành cài đặt...")
        os.system("curl -sL https://tunnelto.dev/install.sh | sh")
        print("✅ Cài đặt Tunnelto hoàn tất!")

    # Chờ cho đến khi ComfyUI mở cổng `port`
    print(f"\n⏳ Đang chờ ComfyUI khởi động trên cổng {port}...")
    while not stop_tunnelto:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        sock.close()
        if result == 0:
            break

    if stop_tunnelto:
        print("\n🛑 Dừng Tunnelto trước khi khởi chạy!")
        return

    print(f"\n✅ ComfyUI đã khởi động! Đang mở Tunnelto trên cổng {port}...\n")

    # Thiết lập API key cho Tunnelto
    os.system(f"{TUNNELTO_PATH} set-auth --key {api[0]}")

    # Chạy Tunnelto trong một tiến trình riêng
    process = threading.Thread(target=lambda: os.system(f"{TUNNELTO_PATH} --subdomain {api[1]} --port {port}"), daemon=True)
    process.start()

    print(f"🔗 Tunnelto đã hoạt động! Truy cập ComfyUI tại: **https://{api[1]}.tunn.dev**")

    # Kiểm tra nếu cần dừng tiến trình Tunnelto
    while not stop_tunnelto:
        time.sleep(1)

    print("\n🛑 Dừng Tunnelto...")
    os.system("pkill -f tunnelto")

def run_comfyui_tunnelto():
    """Chạy ComfyUI thông qua Tunnelto với API Key & Subdomain từ file txt."""
    tunnelto_config_file = os.path.join(CONFIG_DIR, "tunnelto_config.txt")

    # Kiểm tra file cấu hình
    if not os.path.exists(tunnelto_config_file):
        print(f"❌ Không tìm thấy file cấu hình: {tunnelto_config_file}")
        return

    # Đọc API Key & Subdomain từ file
    with open(tunnelto_config_file, "r", encoding="utf-8") as file:
        lines = file.readlines()

    api_key, subdomain = None, None
    for line in lines:
        line = line.strip()
        if line and "-" in line:  # Đảm bảo dòng có dữ liệu và chứa dấu '-'
            try:
                subdomain, api_key = line.split("-", 1)  # Tách subdomain & API Key
                subdomain = subdomain.strip()
                api_key = api_key.strip()
                break  # Lấy dòng đầu tiên hợp lệ
            except ValueError:
                continue  # Bỏ qua dòng lỗi định dạng

    # Kiểm tra giá trị API Key & Subdomain
    if not api_key or not subdomain:
        print("❌ Không thể lấy API Key hoặc Subdomain từ file.")
        return

    print(f"✅ Sử dụng Tunnelto với API Key: {api_key}, Subdomain: {subdomain}")

    # Chạy Tunnelto trong luồng riêng
    threading.Thread(target=tunnelto_thread, daemon=True, args=(COMFYUI_PORT, [api_key, subdomain])).start()

    # Khởi chạy ComfyUI bằng lệnh `!python` để hiển thị log đầy đủ
    print("\n🚀 Đang khởi chạy ComfyUI... Chi tiết quá trình:\n")
    !python main.py --dont-print-server  # Sử dụng os.system() để chạy trực tiếp trong Colab


def run_comfyui_tunnelto_thread():
    """Chạy ComfyUI thông qua Tunnelto với API Key & Subdomain từ file txt."""
    global stop_tunnelto
    stop_tunnelto = False  # Đảm bảo tiến trình chạy bình thường

    tunnelto_config_file = os.path.join(CONFIG_DIR, "tunnelto_config.txt")

    if not os.path.exists(tunnelto_config_file):
        print(f"❌ Không tìm thấy file cấu hình: {tunnelto_config_file}")
        return

    with open(tunnelto_config_file, "r", encoding="utf-8") as file:
        lines = file.readlines()

    api_key, subdomain = None, None
    for line in lines:
        line = line.strip()
        if line and "-" in line:
            try:
                subdomain, api_key = line.split("-", 1)
                subdomain = subdomain.strip()
                api_key = api_key.strip()
                break
            except ValueError:
                continue

    if not api_key or not subdomain:
        print("❌ Không thể lấy API Key hoặc Subdomain từ file.")
        return

    print(f"✅ Sử dụng Tunnelto với API Key: {api_key}, Subdomain: {subdomain}")

    # Chạy Tunnelto trong luồng riêng
    threading.Thread(target=tunnelto_thread_background, daemon=True, args=(COMFYUI_PORT, [api_key, subdomain])).start()

    # Khởi chạy ComfyUI
    print("\n🚀 Đang khởi chạy ComfyUI...")
    os.system("python main.py --dont-print-server")  # Chạy trong Colab

def install_cloudflare():
    """Cài đặt Cloudflare Tunnel nếu chưa có."""
    print("\n🌐 Kiểm tra Cloudflare Tunnel...")
    cloudflared_path = "/usr/local/bin/cloudflared"

    if not os.path.exists(cloudflared_path):
        print("🚀 Cài đặt Cloudflare Tunnel...")
        subprocess.run(["wget", f"https://github.com/cloudflare/cloudflared/releases/latest/download/{CLOUDFLARE_DEB}"])
        subprocess.run(["dpkg", "-i", CLOUDFLARE_DEB])
        print("✅ Cloudflare Tunnel đã được cài đặt!")
    else:
        print("✅ Cloudflare Tunnel đã có sẵn!")

def iframe_thread(port):
    """Kiểm tra khi ComfyUI đã khởi động xong, sau đó chạy Cloudflare Tunnel."""
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\n✅ ComfyUI đã khởi động xong, đang mở Cloudflare Tunnel...\n")

    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )

    for line in p.stderr:
        l = line.strip()
        if "trycloudflare.com " in l:
            print(f"🌍 URL truy cập ComfyUI: {l[l.find('http'):]}")


def run_comfyui_cloudflare():
    """Khởi chạy ComfyUI với Cloudflare Tunnel và hiển thị URL truy cập."""

    # Đảm bảo Cloudflare Tunnel đã được cài đặt
    install_cloudflare()

    # Chạy Cloudflare Tunnel trong một luồng riêng
    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

    # Khởi chạy ComfyUI
    os.system("python main.py --dont-print-server")

def execute_commands_from_file(file):
    """Thực thi các lệnh từ file txt mà không hiển thị log, chỉ báo lỗi nếu có."""

    # Kiểm tra file có tồn tại không
    if not os.path.isfile(file):
        print(f"❌ Không tìm thấy file lệnh: {file}")
        return

    # Đọc danh sách lệnh từ file
    with open(file, "r", encoding="utf-8") as f:
        commands = f.readlines()

    for command in commands:
        command = command.strip()
        if not command:
            continue  # Bỏ qua dòng trống

        result = subprocess.run(command, shell=True, text=True, capture_output=True)

        # Nếu có lỗi, hiển thị lỗi chi tiết
        if result.returncode != 0:
            print(f"❌ Lỗi khi chạy: {command}")
            print(f"🔻 Chi tiết lỗi:\n{result.stderr}")

    print("\n✅ Hoàn tất thực thi lệnh từ file.")

def download_file(url, save_path, name):
    """Tải file từ URL và lưu vào thư mục mong muốn bằng aria2c."""

    # Kiểm tra & cài đặt aria2c nếu cần
    try:
        subprocess.run(["aria2c", "--version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
    except FileNotFoundError:
        print("⚠️ aria2c chưa được cài đặt! Tiến hành cài đặt...")
        subprocess.run(["apt-get", "install", "-y", "aria2"], check=True)
        print("✅ Cài đặt aria2c thành công!")

    # Đảm bảo thư mục lưu trữ tồn tại
    os.makedirs(save_path, exist_ok=True)

    # Đường dẫn đầy đủ của file đầu ra
    output_file = os.path.join(save_path, name)

    # Kiểm tra nếu file đã tồn tại
    if os.path.exists(output_file):
        print(f"✅ File '{name}' đã tồn tại, bỏ qua tải xuống.")
        return

    print(f"📥 Đang tải file: {name}...")

    # Sử dụng `aria2c` để tải file nhanh hơn
    result = subprocess.run([
        "aria2c", "-c", "-x", "16", "-s", "16", "-k", "1M", "-d", save_path, "-o", name, url
    ], text=True, capture_output=True)

    if result.returncode == 0:
        print(f"✅ Tải xuống thành công: {output_file}")
    else:
        print(f"❌ Lỗi khi tải file: {name}")
        print(f"🔻 Chi tiết lỗi:\n{result.stderr}")

def extract_file(file_path, extract_to):
    """Giải nén file .zip hoặc .tar.gz vào thư mục mong muốn."""

    # Kiểm tra nếu file không tồn tại
    if not os.path.exists(file_path):
        print(f"❌ Không tìm thấy file: {file_path}")
        return

    # Đảm bảo thư mục đích tồn tại
    os.makedirs(extract_to, exist_ok=True)

    # Kiểm tra loại file và giải nén tương ứng
    if file_path.endswith(".zip"):
        print(f"📂 Đang giải nén file ZIP: {file_path}...")
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"✅ Giải nén thành công vào: {extract_to}")

    elif file_path.endswith(".tar.gz") or file_path.endswith(".tgz"):
        print(f"📂 Đang giải nén file TAR.GZ: {file_path}...")
        with tarfile.open(file_path, 'r:gz') as tar_ref:
            tar_ref.extractall(extract_to)
        print(f"✅ Giải nén thành công vào: {extract_to}")

    else:
        print(f"⚠️ Không hỗ trợ định dạng file: {file_path}")

# Chạy các quá trình chính
#clone_repo("https://github.com/sipivn/TA-ComfyUI", CONFIG_DIR)
install_aria2c()
install_comfyui()
install_cuda_dependencies()

os.chdir(CUSTOM_NODES_DIR)
execute_commands_from_file("/content/TAConfig/initial_config.txt")
os.chdir(BASE_DIR)

install_custom_nodes("/content/TAConfig/node_base.txt")